In [176]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import nltk
from plotly.subplots import make_subplots
from textblob import TextBlob
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer 
from nltk import word_tokenize
from langdetect import detect_langs
from wordcloud import WordCloud, STOPWORDS
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [177]:
appStore = pd.read_csv('AppStoreData.csv')
googlePlay = pd.read_csv('PlayStoreData.csv')

We can seperate the reviews by ratings and proceed to analysis the text in each rating category
- reduces runtime or smt
#df.loc[df['column_name'] == some_value]
#can create the function to do so

In [178]:
#filter by score and get review col
asResponse = appStore.review
#gpResponse = googlePlay.loc[googlePlay['score'] == 5].text


In [179]:
print(len(asResponse))

130


In [180]:
#asResponse = appStore.review
#gpResponse = googlePlay.reviews

In [181]:
asResponse = asResponse.values.tolist()
gpResponse = gpResponse.values.tolist()

AttributeError: 'list' object has no attribute 'values'

In [ ]:
print(asResponse)

0      Great banking app with attractive interest rat...
1      A bank like no other, no bank have such amazin...
2      Notice that the drop in interest rate of 0.8% ...
3      Sending money into my GXS account is a breeze ...
4      I have to say that the UI/UX is one of the bes...
                             ...                        
125                                                  Wow
126                                                🚀🚀🚀🚀🚀
127    I was trying to make payment using Paynow Qr c...
128          安全漏洞！下载要小心！即使用SingPass 开户也一样不安全！Bug 导致无法通过！
129    21 Jul 23:\nThe integration on iPad is off. Th...
Name: review, Length: 130, dtype: object


In [ ]:
#wordtokenization
wt_as = [word_tokenize(review) for review in asResponse]
wt_gp = [word_tokenize(review) for review in gpResponse]

In [ ]:
print(wt_as[0])

['Great', 'banking', 'app', 'with', 'attractive', 'interest', 'rates', '!', 'Please', 'allow', 'us', 'to', 'add', 'and/or', 'save', 'payees', 'so', 'we', 'don', '’', 't', 'have', 'to', 'keep', 'typing', 'out', 'UEN', 'numbers', 'or', 'account', 'numbers', '.', 'Would', 'be', 'nice', 'to', 'be', 'able', 'to', 'add', 'the', 'debit', 'card', 'to', 'Apple', 'Pay', 'too', '!', '!']


TFIDF
As tfidf filter for words that are important in each comment, it will be essential to use it to predict the sentiment of the review is using 

In [ ]:
#after word tokenizing each sentance, we can lemma/stem
#ok according to reddit, stop word removal can be harmful if we use nnm like bert or RNNS. So i wont do it for now
#we should use our own stopwords, and to find out which are the stop words, word cloud might come into play
#use tldf?
#fk it ill roll first 

#stop_words = ENGLISH_STOP_WORDS

vect = TfidfVectorizer().fit(asResponse)
vect_tfidf = vect.transform(asResponse)
print(vect_tfidf.toarray())

[[0.         0.         0.         ... 0.         0.         0.24454209]
 [0.         0.         0.         ... 0.         0.10382606 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
nltktoken_as = [nltk.word_tokenize(review) for review in asResponse]
test = nltk.word_tokenize(asResponse[0])

In [ ]:
#nltk.download()

In [ ]:
print(test)

['Great', 'banking', 'app', 'with', 'attractive', 'interest', 'rates', '!', 'Please', 'allow', 'us', 'to', 'add', 'and/or', 'save', 'payees', 'so', 'we', 'don', '’', 't', 'have', 'to', 'keep', 'typing', 'out', 'UEN', 'numbers', 'or', 'account', 'numbers', '.', 'Would', 'be', 'nice', 'to', 'be', 'able', 'to', 'add', 'the', 'debit', 'card', 'to', 'Apple', 'Pay', 'too', '!', '!']


In [ ]:

nltk.pos_tag(test)

[('Great', 'NNP'),
 ('banking', 'NN'),
 ('app', 'NN'),
 ('with', 'IN'),
 ('attractive', 'JJ'),
 ('interest', 'NN'),
 ('rates', 'NNS'),
 ('!', '.'),
 ('Please', 'NNP'),
 ('allow', 'VB'),
 ('us', 'PRP'),
 ('to', 'TO'),
 ('add', 'VB'),
 ('and/or', 'NNS'),
 ('save', 'VBP'),
 ('payees', 'NNS'),
 ('so', 'IN'),
 ('we', 'PRP'),
 ('don', 'VBP'),
 ('’', 'JJ'),
 ('t', 'NNS'),
 ('have', 'VBP'),
 ('to', 'TO'),
 ('keep', 'VB'),
 ('typing', 'VBG'),
 ('out', 'RP'),
 ('UEN', 'NNP'),
 ('numbers', 'NNS'),
 ('or', 'CC'),
 ('account', 'NN'),
 ('numbers', 'NNS'),
 ('.', '.'),
 ('Would', 'MD'),
 ('be', 'VB'),
 ('nice', 'JJ'),
 ('to', 'TO'),
 ('be', 'VB'),
 ('able', 'JJ'),
 ('to', 'TO'),
 ('add', 'VB'),
 ('the', 'DT'),
 ('debit', 'NN'),
 ('card', 'NN'),
 ('to', 'TO'),
 ('Apple', 'NNP'),
 ('Pay', 'NNP'),
 ('too', 'RB'),
 ('!', '.'),
 ('!', '.')]

In [ ]:
#General observation for the entire dataset
#calculate sentiment of sentence
sentiment_scoring = [TextBlob(review) for review in gpResponse] # this is for rating = 5
for val in sentiment_scoring:
    print(val)
    print(val.sentiment.polarity)

Quick transfers to bank acc and fast repayments. Very suitable for short term loans
0.3120833333333334
Ok Good system.
0.6
🥰🥰🥰
0.0
Been using this bank for a while now and didn't face any major problems. What I enjoyed the most is the saving pockets area where we could create different pockets for each saving we want to do and even add goals into it. The 2.68% per annum is added daily (0.01) as long as you have a minimum of 150 in your savings. If not, you'll have to wait till your it accumulates up to 0.01 to be added. There was also cashback when you make payment using the card which I didn't know till recently.
0.14464285714285713
👍👍👍👍
0.0
I don't set. A password in my phone n not allow me to down load this apps
-0.15555555555555556
Very good app
0.9099999999999999
Love GxS bank. Would it be available on Google Pay as well? 😉
0.45
Easy and fast app
0.31666666666666665
Easy loan and easy payment with no hassle.
0.43333333333333335
Very good digital savings bank account that helps gro

Roberta

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
example = "This oatmeal is questionable. It is mushy, soft and just so darn weird. I prefer Quaker oats"
TextBlob(example).sentiment


Sentiment(polarity=-0.3, subjectivity=0.7833333333333333)

In [ ]:
#run for roberta model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
score_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(score_dict)
#returns negative, neutral and positive


{'roberta_neg': 0.93354213, 'roberta_neu': 0.060158957, 'roberta_pos': 0.0062989215}


In [ ]:
#do it for reviews
def roberta_score(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    score_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return score_dict

In [ ]:
#asResponse , gpResponse
res = {}
for i, row in tqdm(googlePlay.iterrows(), total = len(asResponse)):
    text = row['text']
    myid = row["text"]
    res[myid] = roberta_score(text)

288it [00:13, 20.63it/s]                       


In [ ]:
#convert to df
result_df = pd.DataFrame(res).T

In [ ]:
print(result_df.head())

                                                    roberta_neg  roberta_neu  \
cant logging using finger print . previously st...     0.879858     0.107318   
For 1st time registration signup by singpass an...     0.856186     0.137936   
Updated - Kindly ignore my previous rating, and...     0.460258     0.496379   
They unable to process my application. 4x attempt      0.893826     0.099285   
Try logging in after logging out completely. Ca...     0.799691     0.186849   

                                                    roberta_pos  
cant logging using finger print . previously st...     0.012824  
For 1st time registration signup by singpass an...     0.005878  
Updated - Kindly ignore my previous rating, and...     0.043363  
They unable to process my application. 4x attempt      0.006889  
Try logging in after logging out completely. Ca...     0.013459  


using transformer pipeline, question answering

In [ ]:
from transformers import pipeline
sent_pipeline = pipeline("sentiment-analysis")
QA_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
config.json: 100%|██████████| 473/473 [00:00<00:00, 3.79MB/s]
model.safetensors: 100%|██████████| 261M/261M [00:11<00:00, 23.0MB/s] 
tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 43.7kB/s]
vocab.txt: 100%|██████████| 213k/213k [00:00<00:00, 11.2MB/s]
tokenizer.json: 100%|██████████| 436k/436k [00:00<00:00, 3.43MB/s]


In [ ]:
sent_pipeline("this application is sooooo good that i will never ever touch it again")

[{'label': 'POSITIVE', 'score': 0.9997859597206116}]

In [175]:
context = r"The pipeline abstraction is a wrapper around all the other available pipelines. It is instantiated as any other pipeline but can provide additional quality of life"

print(QA_pipeline(question="What is good about this application?", context=context))
print(QA_pipeline(question="What can be improved about the application?", context=context))
print(QA_pipeline(question="What do the application lack", context=context))

{'score': 0.23230485618114471, 'start': 125, 'end': 163, 'answer': 'can provide additional quality of life'}
{'score': 0.46773698925971985, 'start': 148, 'end': 163, 'answer': 'quality of life'}
{'score': 0.3263391852378845, 'start': 137, 'end': 163, 'answer': 'additional quality of life'}


In [173]:
type(context)

str

Death, NER

In [134]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

def analyze_review_with_bert(review):
    # Load BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

    # Tokenize input text
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)

    # Perform inference
    outputs = model(**inputs)

    # Get predicted sentiment
    sentiment_score = torch.argmax(outputs.logits).item()
    sentiment_label = "positive" if sentiment_score == 1 else "negative"

    # Extract features related to savings
    savings_keywords = ['saving', 'savings', 'pockets', 'interest rates']
    savings_features = any(keyword in review.lower() for keyword in savings_keywords)

    if sentiment_label == "positive" and savings_features:
        return "Based on the positive sentiment and mention of savings-related features, consider enhancing the savings feature."
    else:
        return "No specific suggestions based on the provided review."



In [135]:
# Example review
review = "My fav feature is the saving pockets can't wait to build my savings with the good interest rates. amazing user interface!!"

# Analyze the review using BERT
analysis_result = analyze_review_with_bert(review)

# Print the analysis result
print(analysis_result)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Based on the positive sentiment and mention of savings-related features, consider enhancing the savings feature.


In [136]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.cluster import KMeans

def extract_features(texts):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    model.eval()

    features = []
    for text in texts:
        input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(input_ids)
            last_hidden_states = outputs.last_hidden_state
            features.append(last_hidden_states.mean(dim=1).squeeze().numpy())
    
    return np.array(features)

def cluster_texts(features, num_clusters=2):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(features)
    return kmeans.labels_

def extract_benefits_problems(reviews, labels):
    benefits = []
    problems = []
    for review, label in zip(reviews, labels):
        if label == 0:
            problems.append(review)
        else:
            benefits.append(review)
    return benefits, problems


In [137]:

# Example customer reviews
reviews = [
    "Hit with fast reversal when transferring money to this gxs bank, first time in my life a fast transfer didn, Äôt succeed. Googled for recent reviews and social media on this gxs bank. Mostly bad reviews and issues. I don‚Äôt need developer reply to my review here as I‚Äôm absolutely not going to touch this bank anymore. And biggest joke of all, you don‚Äôt even have the most basic banking feature to save payee details. What kind of app development staff and quality is that???.",
    "What a joke! Nothing happens after you input your phone number - nothing!",
    "It‚Äôll be so much, and I don‚Äôt exaggerate when I say this, so much faster for you to travel from one end of Singapore to the other, to open an account at a traditional bank branch. "
    "One piece of advice to the management of GXS Bank - do not launch if you are not ready! You are a joke. Please look at Trust Bank. Simple and easy to use. Account opened in like, 2 mins?",
    "GXS offers seamless, fast and super effective digital services."
    "Never ever I had such experience with other banks. GXS definitely is the bank of future! "
    "You must try it to believe!.",
]

# Extract features from reviews
features = extract_features(reviews)

# Cluster reviews into benefits and problems
labels = cluster_texts(features)
benefits, problems = extract_benefits_problems(reviews, labels)

print("Benefits:")
for benefit in benefits:
    print("-", benefit)

print("\nProblems:")
for problem in problems:
    print("-", problem)





Benefits:
- Hit with fast reversal when transferring money to this gxs bank, first time in my life a fast transfer didn, Äôt succeed. Googled for recent reviews and social media on this gxs bank. Mostly bad reviews and issues. I don‚Äôt need developer reply to my review here as I‚Äôm absolutely not going to touch this bank anymore. And biggest joke of all, you don‚Äôt even have the most basic banking feature to save payee details. What kind of app development staff and quality is that???.
- It‚Äôll be so much, and I don‚Äôt exaggerate when I say this, so much faster for you to travel from one end of Singapore to the other, to open an account at a traditional bank branch. One piece of advice to the management of GXS Bank - do not launch if you are not ready! You are a joke. Please look at Trust Bank. Simple and easy to use. Account opened in like, 2 mins?
- GXS offers seamless, fast and super effective digital services.Never ever I had such experience with other banks. GXS definitely is

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
